In [32]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13

In [33]:
import re 
import sys

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_arroba(lista):
    arroba = "@"
    for i in lista: 
        if arroba in i:
            lista.remove(i)
    return lista
def remove_link(lista):
    link = "/"
    for i in lista: 
        if link in i:
            lista.remove(i)
    return lista

In [34]:
dados = pd.read_excel('Iphone_CDados.xlsx')

In [35]:
dados

,Treinamento,Filtro
0,"@kayllanefer bateria de iphone é uma merda, ja...",1
1,cabo de iphone eh feito pra estragar pqp,1
2,@johnpvx melhor iphone de sempre. não me arrep...,1
3,alguém querendo comprar um cabo/carregador de ...,1
4,to sem carregador já tava sofrendo aqui daí le...,1
...,...,...
328,acho que amanhã lança o iphone 12 👀👀👀,0
329,@pirucaking @paulocoelho não dá pra levar a sé...,0
330,"to querendo comprar o iphone 11pro max, para m...",0
331,triste tirar foto só com iphone 😔 pois é a úni...,0


In [36]:
Relevante = dados["Filtro"].isin(["1"])
Irrelevante = dados["Filtro"].isin(["0"])
Relevante = dados.loc[Relevante, :]
Irrelevante = dados.loc[Irrelevante, :]

In [37]:
#limpando texto e fazendo lista

Treino_Relevante = Relevante["Treinamento"]
Treino_Irrelevante = Irrelevante["Treinamento"]

def make_list(Treino_Relevante):
    
    treinamento_relevante = []

    #fazendo lista
    for i in Treino_Relevante:
        treinamento_relevante.append(i)
    
    coma = "'"
    #transformar em texto
    treinamento_relevante = coma.join(treinamento_relevante)

    #função que limpa
    limpa = cleanup(treinamento_relevante)
    no_emoji = remove_emoji(limpa)

    #lista por palavra
    minusculo = no_emoji.lower()
    palavras_relevantes = minusculo.split()

    lista = remove_arroba(palavras_relevantes)
    lista = remove_link(lista)
    
    return lista

todas_palavras_relevantes = make_list(Treino_Relevante)
todas_palavras_irrelevantes = make_list(Treino_Irrelevante)

In [38]:
#pdSeries
serie_relevante = pd.Series(todas_palavras_relevantes)
serie_irrelevante = pd.Series(todas_palavras_irrelevantes)

In [39]:
#tabelas frequencia relativas
tabela_relevantes = serie_relevante.value_counts(True)
tabela_irrelevantes = serie_irrelevante.value_counts(True)

In [40]:
#P(frase/relevante)
def P_frase_dado_relevante(frase):
    
    prob_frase_dado_relevante = 1
    alpha = 1
    V = 10000
    
    for palavra in frase:
        
        if palavra in tabela_relevantes:
            prob_palavra_dado_relevante = (tabela_relevantes[palavra] + alpha)/(V * alpha)
            
        else:
            
            prob_palavra_dado_relevante = alpha/(alpha*V)
        prob_frase_dado_relevante *= prob_palavra_dado_relevante
        
    return prob_palavra_dado_relevante

#P(frase/irrelevante)
def P_frase_dado_irrelevante(frase):
    
    prob_frase_dado_irrelevante = 1
    alpha = 1
    V = 10000
    
    for palavra in frase:
        
        if palavra in tabela_relevantes:
            prob_palavra_dado_irrelevante = (tabela_irrelevantes[palavra] + alpha)/(V * alpha)
        
        else:
            
            prob_palavra_dado_irrelevante = alpha/(alpha * V)
            
        prob_frase_dado_irrelevante *= prob_palavra_dado_irrelevante
        
    return prob_palavra_dado_irrelevante

    